<a href="https://colab.research.google.com/github/arunsingh22/CaseStudies/blob/master/Naiive%20Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import auc,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from tqdm import tqdm
from collections import Counter

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
data = pd.read_csv("/content/drive/My Drive/preprocessed_data.csv",nrows=40000)
data.columns

Index(['school_state', 'teacher_prefix', 'project_grade_category',
       'teacher_number_of_previously_posted_projects', 'project_is_approved',
       'clean_categories', 'clean_subcategories', 'essay', 'price'],
      dtype='object')

In [ ]:
y = data["project_is_approved"].values
data.project_is_approved.value_counts()
X = data.drop(["project_is_approved"],axis=1)

#### **Splitting the Dataset into Dtrain and Dtest**

In [ ]:
#Splitting the data into train,CV,test.
train_data,test_data,train_label,test_label = train_test_split(X,y,test_size = 0.3,stratify =y)

print()
print("Train dataset: ",train_data.shape)
print("Test dataset: ",test_data.shape)
print()
print(Counter(train_label))



Train dataset:  (28000, 8)
Test dataset:  (12000, 8)

Counter({1: 23705, 0: 4295})


In [ ]:
#SET 1: ENCODING USING THE BOW (ESSAY + PROJECT TITLE) 
# Encoding essay into onehot encoding using Countvectorizer/BOW
vect = CountVectorizer()          
BOW_Train_essay = vect.fit_transform(train_data["essay"].values)
print(vect.vocabulary_)

#using the learned vocabulary we encode the CV + test dataset 
BOW_test_essay = vect.transform(test_data["essay"].values)

#Encoding the categorical features School state
vect = CountVectorizer()
vect.fit(train_data.school_state.values)
train_school_state_OHE = vect.transform(train_data["school_state"].values)
test_school_state_OHE = vect.transform(test_data["school_state"].values)

#Encoding the categorical features teacher prefix
vect = CountVectorizer()
vect.fit(train_data.teacher_prefix.values)
train_teacher_prefix_OHE = vect.transform(train_data["teacher_prefix"].values)
test_teacher_prefix_OHE = vect.transform(test_data["teacher_prefix"].values)

#Encoding the categorical features project_grade_category
vect = CountVectorizer()
vect.fit(train_data.project_grade_category.values)
train_project_grade_category_OHE = vect.transform(train_data["project_grade_category"].values)
test_project_grade_category_OHE = vect.transform(test_data["project_grade_category"].values)

#Encoding the categorical features clean_categories	
vect = CountVectorizer()
vect.fit(train_data.clean_categories.values)
train_clean_categories_OHE = vect.transform(train_data["clean_categories"].values)
test_clean_categories_OHE = vect.transform(test_data["clean_categories"].values)

#Encoding the categorical features clean_subcategories	
vect = CountVectorizer()
vect.fit(train_data.clean_subcategories.values)
train_clean_subcategories_OHE = vect.transform(train_data["clean_subcategories"].values)
test_clean_subcategories_OHE = vect.transform(test_data["clean_subcategories"].values)


{'living': 17373, 'high': 13894, 'poverty': 22413, 'crime': 7260, 'students': 28151, 'odds': 20276, 'stacked': 27605, 'they': 29384, 'face': 11011, 'definite': 7938, 'challenges': 5167, 'lives': 17367, 'education': 9608, 'strive': 28103, 'success': 28326, 'progress': 22880, 'normalcy': 19939, 'as': 2399, 'teacher': 28997, 'promised': 22930, 'would': 32458, 'everything': 10575, 'could': 7031, 'give': 12678, 'tools': 29774, 'feel': 11266, 'literacy': 17329, 'challenge': 5161, 'my': 19398, 'limited': 17253, 'reading': 23675, 'skills': 26635, 'many': 17983, 'parents': 21100, 'struggled': 28129, 'school': 25603, 'lost': 17527, 'aspire': 2446, 'fill': 11422, 'classroom': 5712, 'well': 31953, 'deserved': 8194, 'pads': 20944, 'help': 13780, 'engaged': 10078, 'independent': 14924, 'retaining': 24661, 'phonics': 21692, 'affluent': 1354, 'peers': 21379, 'mastered': 18112, 'years': 32621, 'ago': 1422, 'increasing': 14895, 'stamina': 27646, 'what': 31993, 'all': 1606, 'fun': 12257, 'interactive': 1

##**SET 1: Using Bag of Words for featurization**

In [ ]:
project= pd.read_csv("/content/drive/My Drive/Assignment_2/train_data.csv",nrows=30000)
project_title = project["project_title"].values #extracting the project title from the entire file

#Credits: https://colab.research.google.com/drive/1nUiNc5Cb38nTLjUopfNKgPL6g9jZmPY7#scrollTo=d6RCdphRCyn0
print("^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^")
import re
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

processed_project_title = []
# tqdm is for printing the status bar
for sentance in tqdm(project_title):
    sent = decontracted(sentance)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
    # https://gist.github.com/sebleier/554280
    sent = ' '.join(e for e in sent.split())
    processed_project_title.append(sent.lower().strip())
project_title = processed_project_title

print(project_title)
print("*"*1000)
project_title = pd.Series(project_title) # changing the dtype of project_title
train_project_title,test_project_title = train_test_split(project_title,test_size =0.3)


 22%|██▏       | 6614/30000 [00:00<00:00, 66139.75it/s]

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


100%|██████████| 30000/30000 [00:00<00:00, 69428.75it/s]

['educational support for english learners at home', 'wanted projector for hungry learners', 'soccer equipment for awesome middle school students', 'techie kindergarteners', 'interactive math tools', 'flexible seating for mrs jarvis terrific third graders', 'chromebooks for special education reading program', 'it is the 21st century', 'targeting more success in class', 'just for the love of reading pure pleasure', 'reading changes lives', 'elevating academics and parent rapports through technology', 'building life science experiences', 'everyone deserves to be heard', 'tablets can show us the world', 'making recess active', 'making great leap is with leapfrog', 'technology teaches tomorrow is talents today', 'test time', 'wiggling our way to success', 'magic carpet ride in our library', 'from sitting to standing in the classroom', 'books for budding intellectuals', 'instrumental power conquering steam', 's t e a m challenges science technology engineering art math', 'math masters', 'te

In [ ]:
#BOW encoding of the project Title:
vect = CountVectorizer()
BOW_Train_project_title = vect.fit_transform(train_project_title)
BOW_Test_project_title = vect.transform(test_project_title)

print(BOW_Train_project_title.shape)
print(BOW_Test_project_title.shape)

(21000, 7862)
(9000, 7862)


####**Encoding Numerical Features : Price**

In [ ]:
#Credits :https://colab.research.google.com/drive/1EkYHI-vGKnURqLL_u5LEf3yb0YJBVbZW#scrollTo=rNl4cZNlPSxK&forceEdit=true&sandboxMode=true 
from sklearn.preprocessing import StandardScaler
stdscalar = StandardScaler()
stdscalar.fit(train_data['price'].values.reshape(-1,1))

train_price = stdscalar.transform(train_data['price'].values.reshape(-1,1))
test_price = stdscalar.transform(test_data['price'].values.reshape(-1,1))

print("After vectorizations")
print(train_price.shape)

After vectorizations
(28000, 1)


**Encoding Numerical Features : teacher_number_of_previously_posted_projects**

In [ ]:
#Credits :https://colab.research.google.com/drive/1EkYHI-vGKnURqLL_u5LEf3yb0YJBVbZW#scrollTo=rNl4cZNlPSxK&forceEdit=true&sandboxMode=true 
stdscalar = StandardScaler()
stdscalar.fit(train_data['teacher_number_of_previously_posted_projects'].values.reshape(-1,1))

train_prev_posted_proj = stdscalar.transform(train_data['teacher_number_of_previously_posted_projects'].values.reshape(-1,1))
test_prev_posted_proj = stdscalar.transform(test_data['teacher_number_of_previously_posted_projects'].values.reshape(-1,1))

print("After vectorizations")
print(train_prev_posted_proj.shape)

After vectorizations
(28000, 1)


###**Putting all categorical features togther using hstack for MultinomialNB**

In [ ]:
from scipy.sparse import hstack
cat_train = hstack((train_clean_categories_OHE,train_clean_subcategories_OHE,train_project_grade_category_OHE,train_teacher_prefix_OHE,
                  BOW_Train_essay,BOW_Train_project_title,train_school_state_OHE)).tocsr()

cat_test = hstack((test_clean_categories_OHE,test_clean_subcategories_OHE,test_project_grade_category_OHE,test_teacher_prefix_OHE,
               BOW_test_essay,BOW_Test_project_title,test_school_state_OHE)).tocsr()

print(cat_train.shape)
print(cat_test.shape)

ValueError: ignored

###**Putting all Numerical features togther using hstack for GaussianNB**

In [ ]:
# merge two sparse matrices: https://stackoverflow.com/a/19710648/4084039
from scipy.sparse import hstack

Num_train = hstack((train_prev_posted_proj,train_price)).tocsr()

Num_test = hstack((test_prev_posted_proj,test_price)).tocsr()

print(Num_train.shape)
print(Num_test.shape)

ValueError: ignored

###**Putting all the features togther using hstack with BOW representation**

In [ ]:
# merge two sparse matrices: https://stackoverflow.com/a/19710648/4084039
from scipy.sparse import hstack

train = hstack((train_clean_categories_OHE,train_clean_subcategories_OHE,train_project_grade_category_OHE,train_teacher_prefix_OHE,
                  BOW_Train_essay,BOW_Train_project_title,train_prev_posted_proj,train_price,train_school_state_OHE)).tocsr()

test = hstack((test_clean_categories_OHE,test_clean_subcategories_OHE,test_project_grade_category_OHE,test_teacher_prefix_OHE,
               BOW_test_essay,BOW_Test_project_title,test_prev_posted_proj,test_price,test_school_state_OHE)).tocsr()

print(train.shape)
print(test.shape)

### **Data Balancing using Oversampling Technique**

In [ ]:
#Balancing the dataset
from collections import Counter
from imblearn.over_sampling import RandomOverSampler

print("Before bal train_data.shape: ",train.shape,Counter(train_label))
rnd = RandomOverSampler(0.7)
train_BOW,train_label_BOW =rnd.fit_sample(train,train_label)
print("After bal train_data.shape:{0},{1} ",train_BOW.shape,Counter(train_label_BOW))

### **Model 1: Plotting the curve for NB with BOW**

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
from sklearn.model_selection import RandomizedSearchCV

clf = KNeighborsClassifier(n_jobs=-1)
parameters = {'n_neighbors':range(1,71,2),
              "weights":["distance"],
             }
model = RandomizedSearchCV(clf,param_distributions=parameters, cv=3, scoring='roc_auc',return_train_score=True,verbose=1,)
model.fit(train_BOW, train_label_BOW)

results = pd.DataFrame.from_dict(model.cv_results_)
results = results.sort_values(['param_n_neighbors'])

train_auc= results['mean_train_score']
train_auc_std= results['std_train_score']
cv_auc = results['mean_test_score'] 
cv_auc_std= results['std_test_score']
K =  results['param_n_neighbors']

plt.plot(K, train_auc, label='Train AUC')
plt.plot(K, cv_auc, label='CV AUC')
plt.scatter(K, train_auc, label='Train AUC points')
plt.scatter(K, cv_auc, label='CV AUC points')
plt.legend()
plt.xlabel("K: hyperparameter")
plt.ylabel("Mean Test Score")
plt.title("Hyper parameter Vs Mean Test Score")
plt.grid()
plt.show()
print(model.best_score_)
print(model.best_params_)

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_curve.html#sklearn.metrics.roc_curve
from sklearn.metrics import roc_curve, auc
from sklearn.calibration import CalibratedClassifierCV
best_k=69
clf = KNeighborsClassifier(n_neighbors=best_k,weights="distance")
# model1.fit(train,train_label_BOW)
# calibrate model on validation data
model1 = CalibratedClassifierCV(clf, cv=3)
model1.fit(train_BOW,train_label_BOW)

In [ ]:
y_train_pred = model1.predict_proba(train_BOW)    
y_test_pred = model1.predict_proba(test) 
print(y_train_pred.shape)

train_fpr, train_tpr, tr_thresholds = roc_curve(train_label_BOW, y_train_pred[:,1])
test_fpr, test_tpr, te_thresholds = roc_curve(test_label, y_test_pred[:,1])

plt.plot(train_fpr, train_tpr, label="train AUC ="+str(auc(train_fpr, train_tpr)))
plt.plot(test_fpr, test_tpr, label="test AUC ="+str(auc(test_fpr, test_tpr)))
plt.legend()
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("ROC_AUC CURVE")
plt.grid()
plt.show()

print()
#Prediction
prediction=model1.predict(test)
#evaluation(Accuracy)
print("Accuracy:",accuracy_score(prediction,test_label))
print()
#evaluation(Confusion Metrix)
print("Confusion Metrix:\n",confusion_matrix(prediction,test_label))

##**Model 2: TF-IDF features for NB**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Encoding the eassay column 
vect = TfidfVectorizer()
train_tfidf_essay =vect.fit_transform(train_data["essay"].values)
print(train_tfidf_essay.shape)

#encodng the same for test essay column also.
test_tfidf_essay =vect.transform(test_data["essay"].values)
print(test_tfidf_essay.shape)

# Encoding the project_title column 
vect = TfidfVectorizer()
train_tfidf_project_title =vect.fit_transform(train_project_title)
print(train_tfidf_project_title.shape)
#encodng the same for test essay column also.
test_tfidf_project_title =vect.transform(test_project_title)
print(test_tfidf_project_title.shape)


In [ ]:
#KNN training of new feature set using TF-IDF
# merge two sparse matrices: https://stackoverflow.com/a/19710648/4084039
from scipy.sparse import hstack

train_tfidf = hstack((train_clean_categories_OHE,train_clean_subcategories_OHE,train_project_grade_category_OHE,train_teacher_prefix_OHE,
                  train_tfidf_essay,train_tfidf_project_title,train_prev_posted_proj,train_price,train_school_state_OHE)).tocsr()

test_tfidf = hstack((test_clean_categories_OHE,test_clean_subcategories_OHE,test_project_grade_category_OHE,test_teacher_prefix_OHE,
                    test_tfidf_essay,test_tfidf_project_title,test_prev_posted_proj,test_price,test_school_state_OHE)).tocsr()

print(train_tfidf.shape)
print(test_tfidf.shape)

### **Data Balancing using Oversampling Technique**

In [ ]:
#Balancing the dataset 
from collections import Counter
from imblearn.over_sampling import RandomOverSampler

print("Before bal train_data: ",train_tfidf.shape)
print(Counter(train_label))
rnd = RandomOverSampler(0.7)
train_tfidf,train_label_tfidf=rnd.fit_sample(train_tfidf,train_label)
print()
print("After bal train_data.shape:{0},{1} ",train_tfidf.shape,Counter(train_label))

### **Ploting curve for KNN with Tf-Idf**

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
# from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

model = KNeighborsClassifier(n_jobs=-1)
parameters = {'n_neighbors':range(1,71,2),
              "weights":["distance"],
             }
clf = RandomizedSearchCV(model, parameters, cv=3, scoring='roc_auc',return_train_score=True,verbose=1,random_state=99)
clf.fit(train_tfidf, train_label_tfidf)

results = pd.DataFrame.from_dict(clf.cv_results_)
results = results.sort_values(['param_n_neighbors'])

train_auc= results['mean_train_score']
train_auc_std= results['std_train_score']
cv_auc = results['mean_test_score'] 
cv_auc_std= results['std_test_score']
K =  results['param_n_neighbors']

plt.plot(K, train_auc, label='Train AUC')
plt.plot(K, cv_auc, label='CV AUC')
plt.scatter(K, train_auc, label='Train AUC points')
plt.scatter(K, cv_auc, label='CV AUC points')

plt.legend()
plt.xlabel("K: hyperparameter")
plt.ylabel("Mean Test Score")
plt.title("Hyper parameter Vs Mean Test Score")
plt.grid()
plt.show()
print(clf.best_params_)


In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_curve.html#sklearn.metrics.roc_curve
from sklearn.metrics import roc_curve, auc
from sklearn.calibration import CalibratedClassifierCV
best_k=65

clf = KNeighborsClassifier(n_neighbors=best_k,weights="distance")

model1 = CalibratedClassifierCV(clf,cv=3)
model1.fit(train_tfidf,train_label_tfidf)

y_train_pred = model1.predict_proba(train_tfidf)    
y_test_pred = model1.predict_proba(test_tfidf) 


In [ ]:

train_fpr, train_tpr, tr_thresholds = roc_curve(train_label_tfidf, y_train_pred[:,1])
test_fpr, test_tpr, te_thresholds = roc_curve(test_label, y_test_pred[:,1])
K =  results['param_n_neighbors']
plt.plot(train_fpr, train_tpr, label="train AUC ="+str(auc(train_fpr, train_tpr)))
plt.plot(test_fpr, test_tpr, label="test AUC ="+str(auc(test_fpr, test_tpr)))
plt.legend()

plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("ROC_AUC Score")
plt.grid()
plt.show()

print()
#Prediction
prediction=model1.predict(test_tfidf)
#evaluation(Accuracy)
print("Accuracy:",accuracy_score(prediction,test_label))
print()
#evaluation(Confusion Metrix)
print("Confusion Metrix:\n",confusion_matrix(prediction,test_label))

In [ ]:
from prettytable import PrettyTable    
x = PrettyTable()

x.field_names = ["Model #", "Vectorizer", "Best Alpha", "AUC score"]

x.add_row(["Model 1.0", "BOW",69 , 0.634])
x.add_row(["Model 2.0", "Tfidf", 49, 0.46])
print(x)
